In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
dir_path = './data/'

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [3]:
cid = pd.read_excel('aladin_Category_CID_20210927.xls')
cid = cid[['CID', '몰', '카테고리명',  '1Depth', '2Depth']]
cid.columns = ['CID', 'mall', 'category_name',  '1Depth', '2Depth']
cid

,CID,mall,category_name,1Depth,2Depth
0,1230,국내도서,가정/요리/뷰티,가정/요리/뷰티,NaN
1,90452,국내도서,가계부,가정/요리/뷰티,가계부
2,53471,국내도서,건강요리,가정/요리/뷰티,건강요리
3,53489,국내도서,결혼/가족,가정/요리/뷰티,결혼/가족
4,53513,국내도서,가정의례,가정/요리/뷰티,결혼/가족
...,...,...,...,...,...
19581,40697,전자책,임신/출산,Gift,특별한 날
19582,40698,전자책,입학/졸업,Gift,특별한 날
19583,40730,전자책,직접 선물 만들기,Gift,특별한 날
19584,41004,전자책,추석,Gift,특별한 날


In [4]:
cid['CID']

0         1230
1        90452
2        53471
3        53489
4        53513
         ...  
19581    40697
19582    40698
19583    40730
19584    41004
19585    41075
Name: CID, Length: 19586, dtype: int64

In [5]:
import os
import sys
import requests
import json

#키 정의
key = 'ttbsmk9805101528002'

## 1.1. Best seller(Book)

In [6]:
book_list1 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=Bestseller&SearchTarget=Book" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list1.append(book_dict)
best_book = pd.DataFrame(book_list1)
best_book = best_book.drop_duplicates()

In [7]:
best_df = pd.merge(best_book, cid, how='left', left_on='CID', right_on='CID')
best_df

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,,316015232,주술회전 22 (초판한정 스페셜 일러스트 카드 + 선착순 한정 옷코츠 유타 포토 카...,"사쿠라지마 콜로니에 갑자기 날아든 의문의 주령! 그것은 사후에 저주로 돌변한, 마키...","아쿠타미 게게 (지은이), 이정운 (옮긴이)",234160,4133,10,5400,6000,2023-05-31,서울미디어코믹스(서울문화사),국내도서,액션 판타지,만화,본격장르만화
1,9791168473690,309295168,세이노의 가르침,"2000년부터 발표된 그의 주옥같은 글들. 독자들이 자발적으로 만든 제본서는 물론,...",세이노 (지은이),846030,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
2,9791168415232,315275569,흔한남매 13,"으뜸이와 에이미의 서프라이즈 장난 대결, 소풍 도시락 싸기 대작전, 신기하고 오싹한...","흔한남매 (지은이), 백난도 (글), 유난희 (그림), 흔한컴퍼니 (감수)",112480,49995,10,13050,14500,2023-04-25,미래엔아이세움,국내도서,만화 일반,어린이,TV/만화/영화
3,9791190299770,313814762,모든 삶은 흐른다,2022년 프랑스 최고의 철학과 교수로 꼽힌 로랑스 드빌레르의 인문에세이로 출간 후...,"로랑스 드빌레르 (지은이), 이주영 (옮긴이)",73175,51378,10,15120,16800,2023-04-03,FIKA(피카),국내도서,교양 인문학,인문학,교양 인문학
4,9791188331888,313250109,사장학개론,"한국과 미국, 전 세계를 오가며 ‘사장을 가르치는 사장’으로 알려진 『돈의 속성』의...",김승호 (지은이),137645,3103,9,22500,25000,2023-04-19,스노우폭스북스,국내도서,경영 일반,경제경영,기업 경영
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9791169990882,313708493,2023 해커스 투자자산운용사 최종핵심정리문제집 + 실전모의고사 3회분 - 2023...,"핵심정리부터 실전까지 한 번에 완성할 수 있으며, 시험에 나올 내용만 모아 쉽고 빠...","백영, 송현남, 장민기, 조중식, 해커스 금융아카데미 (지은이)",3920,181648,10,31410,34900,2023-03-30,해커스금융,국내도서,투자자산운용사,수험서/자격증,민간자격
996,9791169518260,315236340,나는 내 인생이 참 좋다 - 매일 새로 만나는 생의 찬란함에 관하여,인생 전반을 차례로 되짚으며 힘겹고 막막하고 고통스러운 순간에도 어떻게 삶의 긍정적...,"메리 파이퍼 (지은이), 허윤정 (옮긴이)",3300,51381,9,16200,18000,2023-04-21,티라미수 더북,국내도서,인문 에세이,인문학,인문 에세이
997,9791136767073,315606839,"귀여운 여자아이에게 공략당하는 건 좋아하세요? 5 - J Novel, 완결","“나……, 좋아해?” 서로의 호감을 드러내는 명확한 증거를 보게 된 미카도와 키사....","아마노 세이주 (지은이), 이엽 (옮긴이), kakao (일러스트)",3090,51105,10,6300,7000,2023-04-26,서울문화사,국내도서,제이(J) 노벨,소설/시/희곡,라이트 노벨
998,9791138478359,315751058,성추행당할 뻔한 S급 미소녀를 구해주고 보니 옆자리 소꿉친구였다 5 - S Novel+,학교에서 S급 미소녀라고 소문이 자자한 후시미 히나를 주역으로 삼은 독립 영화 제작...,"켄노지 (지은이), 플라이 (그림), 천선필 (옮긴이)",2610,89167,0,8820,9800,2023-05-10,㈜소미미디어,국내도서,S 노벨,소설/시/희곡,라이트 노벨


## 1.2. Best seller(e-book)

In [8]:
book_list2 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=Bestseller&SearchTarget=eBook" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list2.append(book_dict)
best_e_book = pd.DataFrame(book_list2)
best_e_book = best_e_book.drop_duplicates()

In [9]:
best_e_df = pd.merge(best_e_book, cid, how='left', left_on='CID', right_on='CID')
best_e_df

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9791140508143,315738712,[BL] 개의 가면 1,,아르곤18 (지은이),23110,127464,10,3400,3400,2023-03-22,고렘팩토리,전자책,국내,BL,BL소설
1,9791170571506,315791605,[루비] 섹프는 역시 없던 일로! (한정판),,베니 (지은이),16200,127587,10,4410,4900,2023-04-28,현대지능개발사(ruvill),전자책,국외,BL,BL만화
2,9791137339101,315595573,[고화질] 헌터X헌터 신장판 37,"왕위 계승전, 그 동안의 이야기. 배 안에서 시작된 카킨왕국의 왕위 계승전. 크라피...",토가시 요시히로 (지은이),15090,40332,10,2700,3000,2023-04-25,학산문화사/DCW,전자책,액션만화,만화,소년만화
3,9791141108205,315791641,[고화질] 새벽의 연화 38,,"미즈호 쿠사나기 (지은이), 이상은 (옮긴이)",10520,40411,10,2500,2500,2023-04-28,학산문화사,전자책,판타지,만화,본격 장르만화
4,9791169681872,315545405,[BL] 메테오(METEOR) 1,,육손이 (지은이),16550,127464,9,3000,3000,2023-04-24,라피스,전자책,국내,BL,BL소설
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9791133698295,47371496,[고화질세트] 데스노트 (총12권/완결),,"오바 츠구미 (지은이), 오바타 다케시 (그림)",2871,40114,10,36000,36000,2006-12-01,대원씨아이(만화),전자책,추리/미스터리,만화,본격 장르만화
996,9788927095828,294674916,[루비] 안기고 싶은 남자 1위에게 협박당하고 있습니다 8부 - 뉴 루비코믹스 2730,"드라마 “clear""의 촬영을 계기로 고즈바라파의 주목을 받게 된 타카토. 촬영 종...",사쿠라비 하시고 (지은이),2869,127587,9,3500,3500,2022-05-13,현대지능개발사(ruvill),전자책,국외,BL,BL만화
997,9791138415798,308270200,어서 오세요 실력지상주의 교실에 2학년 편 07 - S Novel,체육대회가 끝나고 고도 육성 고등학교 첫 문화제가 다가왔다. 반 아이들에게 벽을 쌓...,"키누가사 쇼고 (지은이), 토모세 슌사쿠 (그림), 조민정 (옮긴이)",2795,164320,10,5000,5000,2023-01-04,㈜소미미디어,전자책,국외,라이트 노벨,국외
998,9791190669054,287249071,삼국지 원전 완역판 1 도원,삼국지는 중국의 후한 말부터 100여 년간 이어진 영웅호걸들의 흥망치란을 한데 담아...,"요시카와 에이지 (엮은이), 나관중 (원작), 바른번역 (옮긴이)",2735,56544,9,900,1000,2020-03-09,참돌,전자책,중국소설,소설/시/희곡,중국소설


## 1.3. Best seller(Book + eBook)

In [10]:
best = pd.concat((best_df, best_e_df))
best = best.drop_duplicates()
best.index = [i for i in range(len(best.index))]
best

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,,316015232,주술회전 22 (초판한정 스페셜 일러스트 카드 + 선착순 한정 옷코츠 유타 포토 카...,"사쿠라지마 콜로니에 갑자기 날아든 의문의 주령! 그것은 사후에 저주로 돌변한, 마키...","아쿠타미 게게 (지은이), 이정운 (옮긴이)",234160,4133,10,5400,6000,2023-05-31,서울미디어코믹스(서울문화사),국내도서,액션 판타지,만화,본격장르만화
1,9791168473690,309295168,세이노의 가르침,"2000년부터 발표된 그의 주옥같은 글들. 독자들이 자발적으로 만든 제본서는 물론,...",세이노 (지은이),846030,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
2,9791168415232,315275569,흔한남매 13,"으뜸이와 에이미의 서프라이즈 장난 대결, 소풍 도시락 싸기 대작전, 신기하고 오싹한...","흔한남매 (지은이), 백난도 (글), 유난희 (그림), 흔한컴퍼니 (감수)",112480,49995,10,13050,14500,2023-04-25,미래엔아이세움,국내도서,만화 일반,어린이,TV/만화/영화
3,9791190299770,313814762,모든 삶은 흐른다,2022년 프랑스 최고의 철학과 교수로 꼽힌 로랑스 드빌레르의 인문에세이로 출간 후...,"로랑스 드빌레르 (지은이), 이주영 (옮긴이)",73175,51378,10,15120,16800,2023-04-03,FIKA(피카),국내도서,교양 인문학,인문학,교양 인문학
4,9791188331888,313250109,사장학개론,"한국과 미국, 전 세계를 오가며 ‘사장을 가르치는 사장’으로 알려진 『돈의 속성』의...",김승호 (지은이),137645,3103,9,22500,25000,2023-04-19,스노우폭스북스,국내도서,경영 일반,경제경영,기업 경영
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9791133698295,47371496,[고화질세트] 데스노트 (총12권/완결),,"오바 츠구미 (지은이), 오바타 다케시 (그림)",2871,40114,10,36000,36000,2006-12-01,대원씨아이(만화),전자책,추리/미스터리,만화,본격 장르만화
1996,9788927095828,294674916,[루비] 안기고 싶은 남자 1위에게 협박당하고 있습니다 8부 - 뉴 루비코믹스 2730,"드라마 “clear""의 촬영을 계기로 고즈바라파의 주목을 받게 된 타카토. 촬영 종...",사쿠라비 하시고 (지은이),2869,127587,9,3500,3500,2022-05-13,현대지능개발사(ruvill),전자책,국외,BL,BL만화
1997,9791138415798,308270200,어서 오세요 실력지상주의 교실에 2학년 편 07 - S Novel,체육대회가 끝나고 고도 육성 고등학교 첫 문화제가 다가왔다. 반 아이들에게 벽을 쌓...,"키누가사 쇼고 (지은이), 토모세 슌사쿠 (그림), 조민정 (옮긴이)",2795,164320,10,5000,5000,2023-01-04,㈜소미미디어,전자책,국외,라이트 노벨,국외
1998,9791190669054,287249071,삼국지 원전 완역판 1 도원,삼국지는 중국의 후한 말부터 100여 년간 이어진 영웅호걸들의 흥망치란을 한데 담아...,"요시카와 에이지 (엮은이), 나관중 (원작), 바른번역 (옮긴이)",2735,56544,9,900,1000,2020-03-09,참돌,전자책,중국소설,소설/시/희곡,중국소설


In [11]:
best = best[best['customer_review_rank'] != 0]
best.index = [i for i in range(1, len(best.index) + 1)]
best

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,,316015232,주술회전 22 (초판한정 스페셜 일러스트 카드 + 선착순 한정 옷코츠 유타 포토 카...,"사쿠라지마 콜로니에 갑자기 날아든 의문의 주령! 그것은 사후에 저주로 돌변한, 마키...","아쿠타미 게게 (지은이), 이정운 (옮긴이)",234160,4133,10,5400,6000,2023-05-31,서울미디어코믹스(서울문화사),국내도서,액션 판타지,만화,본격장르만화
2,9791168473690,309295168,세이노의 가르침,"2000년부터 발표된 그의 주옥같은 글들. 독자들이 자발적으로 만든 제본서는 물론,...",세이노 (지은이),846030,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
3,9791168415232,315275569,흔한남매 13,"으뜸이와 에이미의 서프라이즈 장난 대결, 소풍 도시락 싸기 대작전, 신기하고 오싹한...","흔한남매 (지은이), 백난도 (글), 유난희 (그림), 흔한컴퍼니 (감수)",112480,49995,10,13050,14500,2023-04-25,미래엔아이세움,국내도서,만화 일반,어린이,TV/만화/영화
4,9791190299770,313814762,모든 삶은 흐른다,2022년 프랑스 최고의 철학과 교수로 꼽힌 로랑스 드빌레르의 인문에세이로 출간 후...,"로랑스 드빌레르 (지은이), 이주영 (옮긴이)",73175,51378,10,15120,16800,2023-04-03,FIKA(피카),국내도서,교양 인문학,인문학,교양 인문학
5,9791188331888,313250109,사장학개론,"한국과 미국, 전 세계를 오가며 ‘사장을 가르치는 사장’으로 알려진 『돈의 속성』의...",김승호 (지은이),137645,3103,9,22500,25000,2023-04-19,스노우폭스북스,국내도서,경영 일반,경제경영,기업 경영
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,9791133698295,47371496,[고화질세트] 데스노트 (총12권/완결),,"오바 츠구미 (지은이), 오바타 다케시 (그림)",2871,40114,10,36000,36000,2006-12-01,대원씨아이(만화),전자책,추리/미스터리,만화,본격 장르만화
1658,9788927095828,294674916,[루비] 안기고 싶은 남자 1위에게 협박당하고 있습니다 8부 - 뉴 루비코믹스 2730,"드라마 “clear""의 촬영을 계기로 고즈바라파의 주목을 받게 된 타카토. 촬영 종...",사쿠라비 하시고 (지은이),2869,127587,9,3500,3500,2022-05-13,현대지능개발사(ruvill),전자책,국외,BL,BL만화
1659,9791138415798,308270200,어서 오세요 실력지상주의 교실에 2학년 편 07 - S Novel,체육대회가 끝나고 고도 육성 고등학교 첫 문화제가 다가왔다. 반 아이들에게 벽을 쌓...,"키누가사 쇼고 (지은이), 토모세 슌사쿠 (그림), 조민정 (옮긴이)",2795,164320,10,5000,5000,2023-01-04,㈜소미미디어,전자책,국외,라이트 노벨,국외
1660,9791190669054,287249071,삼국지 원전 완역판 1 도원,삼국지는 중국의 후한 말부터 100여 년간 이어진 영웅호걸들의 흥망치란을 한데 담아...,"요시카와 에이지 (엮은이), 나관중 (원작), 바른번역 (옮긴이)",2735,56544,9,900,1000,2020-03-09,참돌,전자책,중국소설,소설/시/희곡,중국소설


## 2.1. blog best(Book)

In [12]:
book_list3 = []

for i in range(1, 5):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=BlogBest&SearchTarget=Book" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list3.append(book_dict)
blog_book = pd.DataFrame(book_list3)
blog_book = blog_book.drop_duplicates()

In [13]:
blog_df = pd.merge(blog_book, cid, how='left', left_on='CID', right_on='CID')
blog_df

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9788932923086,314130432,우리 슬픔의 거울,현존하는 가장 재미있는 거장이자 공쿠르상 수상 작가인 피에르 르메트르가 내놓은 신작...,"피에르 르메트르 (지은이), 임호경 (옮긴이)",4855,50921,10,16920,18800,2023-04-10,열린책들,국내도서,프랑스소설,소설/시/희곡,프랑스소설
1,9791197918162,315010555,수치 - 방대하지만 단일하지 않은 성폭력의 역사,"전 세계적으로 난무하는 성적 폭력을 일으키고 계속되도록 만든 이념과 제도, 법적 틀...","조애나 버크 (지은이), 송은주 (옮긴이), 정희진 (해제)",3120,51075,9,24300,27000,2023-04-28,디플롯,국내도서,성차별/성폭력문제,사회과학,사회문제
2,9791168126121,314433265,퍼핏 쇼,영국 컴브리아 지역의 선사 유물 ‘환상열석’에서 발견된 불에 타 죽은 시신들. 언론...,"M. W. 크레이븐 (지은이), 김해온 (옮긴이)",8960,51062,9,16200,18000,2023-04-06,위즈덤하우스,국내도서,영미 추리/미스터리소설,소설/시/희곡,추리/미스터리소설
3,9791156754107,314440287,"자신의 존재에 대해 사과하지 말 것 - 삶, 사랑, 관계에 닿기 위한 자폐인 과학자...","여덟 살에 자폐스펙트럼장애를 진단받고, 오랜시간 ADHD, 범불안장애, 강박장애, ...","카밀라 팡 (지은이), 김보은 (옮긴이)",30120,51002,10,16920,18800,2023-04-12,푸른숲,국내도서,기초과학/교양과학,과학,기초과학/교양과학
4,9788925576633,315292775,블랙 쇼맨과 환상의 여자,"미스터리의 제왕이 돌아왔다. 이번 신간은 한층 속도감을 더한 소설집으로, 전작 《블...","히가시노 게이고 (지은이), 최고은 (옮긴이)",32990,51058,9,16200,18000,2023-04-25,알에이치코리아(RHK),국내도서,일본 추리/미스터리소설,소설/시/희곡,추리/미스터리소설
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,9791190955911,313959213,같이 가면 길이 된다,국제노동기구(ILO)에서 고용정책국장으로 일하는 이상헌이 치열한 숙고와 엄격한 응시...,이상헌 (지은이),12085,51304,8,16200,18000,2023-04-07,생각의힘,국내도서,사회학 일반,사회과학,사회학
137,9791160409932,315681393,별일은 없고요?,2012년 작품 활동을 시작한 이래 소리 높여 주장하기보다 온화하게 스며드는 특유의...,이주란 (지은이),3820,50993,10,13500,15000,2023-04-30,한겨레출판,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
138,9788937429910,314238593,수많은 운명의 집,"오스트리아를 대표하는 작가이자 유럽의 지성으로 불리며 빼어난 전기 작품과 소설, 아...","슈테판 츠바이크 (지은이), 이미선 (옮긴이)",2955,51373,0,10620,11800,2023-03-31,민음사,국내도서,외국에세이,에세이,외국에세이
139,9791159319181,314894871,헤븐,이 시대를 살아가는 청소년들이 겪는 문제를 예리하게 포착해 간결한 문체로 그려냄으로...,"가와카미 미에코 (지은이), 이지수 (옮긴이)",870,50998,9,13320,14800,2023-04-28,책세상,국내도서,1950년대 이후 일본소설,소설/시/희곡,일본소설


In [14]:
blog_df = blog_df[blog_df['customer_review_rank'] != 0]
blog_df.index = [i for i in range(1, len(blog_df.index) + 1)]
blog_df

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9788932923086,314130432,우리 슬픔의 거울,현존하는 가장 재미있는 거장이자 공쿠르상 수상 작가인 피에르 르메트르가 내놓은 신작...,"피에르 르메트르 (지은이), 임호경 (옮긴이)",4855,50921,10,16920,18800,2023-04-10,열린책들,국내도서,프랑스소설,소설/시/희곡,프랑스소설
2,9791197918162,315010555,수치 - 방대하지만 단일하지 않은 성폭력의 역사,"전 세계적으로 난무하는 성적 폭력을 일으키고 계속되도록 만든 이념과 제도, 법적 틀...","조애나 버크 (지은이), 송은주 (옮긴이), 정희진 (해제)",3120,51075,9,24300,27000,2023-04-28,디플롯,국내도서,성차별/성폭력문제,사회과학,사회문제
3,9791168126121,314433265,퍼핏 쇼,영국 컴브리아 지역의 선사 유물 ‘환상열석’에서 발견된 불에 타 죽은 시신들. 언론...,"M. W. 크레이븐 (지은이), 김해온 (옮긴이)",8960,51062,9,16200,18000,2023-04-06,위즈덤하우스,국내도서,영미 추리/미스터리소설,소설/시/희곡,추리/미스터리소설
4,9791156754107,314440287,"자신의 존재에 대해 사과하지 말 것 - 삶, 사랑, 관계에 닿기 위한 자폐인 과학자...","여덟 살에 자폐스펙트럼장애를 진단받고, 오랜시간 ADHD, 범불안장애, 강박장애, ...","카밀라 팡 (지은이), 김보은 (옮긴이)",30120,51002,10,16920,18800,2023-04-12,푸른숲,국내도서,기초과학/교양과학,과학,기초과학/교양과학
5,9788925576633,315292775,블랙 쇼맨과 환상의 여자,"미스터리의 제왕이 돌아왔다. 이번 신간은 한층 속도감을 더한 소설집으로, 전작 《블...","히가시노 게이고 (지은이), 최고은 (옮긴이)",32990,51058,9,16200,18000,2023-04-25,알에이치코리아(RHK),국내도서,일본 추리/미스터리소설,소설/시/희곡,추리/미스터리소설
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,9788937475481,184148396,망할 놈의 예술을 한답시고,민음사 세계시인선 리뉴얼판. 찰스 부코스키가 노년에 마지막으로 출간한 대표작 &lt...,"찰스 부코스키 (지은이), 황소연 (옮긴이)",3317,51168,9,11700,13000,2019-02-22,민음사,국내도서,외국시,소설/시/희곡,시
117,9791188331888,313250109,사장학개론,"한국과 미국, 전 세계를 오가며 ‘사장을 가르치는 사장’으로 알려진 『돈의 속성』의...",김승호 (지은이),137645,3103,9,22500,25000,2023-04-19,스노우폭스북스,국내도서,경영 일반,경제경영,기업 경영
118,9791190955911,313959213,같이 가면 길이 된다,국제노동기구(ILO)에서 고용정책국장으로 일하는 이상헌이 치열한 숙고와 엄격한 응시...,이상헌 (지은이),12085,51304,8,16200,18000,2023-04-07,생각의힘,국내도서,사회학 일반,사회과학,사회학
119,9791160409932,315681393,별일은 없고요?,2012년 작품 활동을 시작한 이래 소리 높여 주장하기보다 온화하게 스며드는 특유의...,이주란 (지은이),3820,50993,10,13500,15000,2023-04-30,한겨레출판,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설


## 3.1. Best + Blog

In [15]:
best_blog = pd.concat((best, blog_df))
best_blog = best_blog.drop_duplicates()
best_blog.index = [i for i in range(len(best_blog.index))]
best_blog

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,,316015232,주술회전 22 (초판한정 스페셜 일러스트 카드 + 선착순 한정 옷코츠 유타 포토 카...,"사쿠라지마 콜로니에 갑자기 날아든 의문의 주령! 그것은 사후에 저주로 돌변한, 마키...","아쿠타미 게게 (지은이), 이정운 (옮긴이)",234160,4133,10,5400,6000,2023-05-31,서울미디어코믹스(서울문화사),국내도서,액션 판타지,만화,본격장르만화
1,9791168473690,309295168,세이노의 가르침,"2000년부터 발표된 그의 주옥같은 글들. 독자들이 자발적으로 만든 제본서는 물론,...",세이노 (지은이),846030,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
2,9791168415232,315275569,흔한남매 13,"으뜸이와 에이미의 서프라이즈 장난 대결, 소풍 도시락 싸기 대작전, 신기하고 오싹한...","흔한남매 (지은이), 백난도 (글), 유난희 (그림), 흔한컴퍼니 (감수)",112480,49995,10,13050,14500,2023-04-25,미래엔아이세움,국내도서,만화 일반,어린이,TV/만화/영화
3,9791190299770,313814762,모든 삶은 흐른다,2022년 프랑스 최고의 철학과 교수로 꼽힌 로랑스 드빌레르의 인문에세이로 출간 후...,"로랑스 드빌레르 (지은이), 이주영 (옮긴이)",73175,51378,10,15120,16800,2023-04-03,FIKA(피카),국내도서,교양 인문학,인문학,교양 인문학
4,9791188331888,313250109,사장학개론,"한국과 미국, 전 세계를 오가며 ‘사장을 가르치는 사장’으로 알려진 『돈의 속성』의...",김승호 (지은이),137645,3103,9,22500,25000,2023-04-19,스노우폭스북스,국내도서,경영 일반,경제경영,기업 경영
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,9791188519422,314774017,나의 가치를 높여주는 대화법 - 성공하는 사람들을 위한 72가지 화술 법칙,"단순한 의사 표현 방법을 넘어 상대의 심리를 파악하면서 효과적으로 마음을 전하고, ...",안은표 (지은이),140,70227,10,16020,17800,2023-04-18,시아출판사,국내도서,화술,자기계발,협상/설득/화술
1721,9791192488301,314995019,날지 않는 꿈도 괜찮아 - 내적 성장을 위한 지친 마음 다스리기,"수십 년간 국내외를 오가며 상처받은 수만 명의 마음을 돌봐온 미술치료계의 일등공신,...",김선현 (지은이),2600,70216,9,16650,18500,2023-04-14,베가북스,국내도서,성공학,자기계발,성공
1722,9788984078895,314796682,"교육이 없는 나라 - 서열화된 대학, 경쟁력 없는 교육, 불행한 사회",저자는 모든 교육 문제의 출발점은 고3까지만 쓸데없이 어렵게 공부하고 이후는 학습 ...,이승섭 (지은이),1750,51090,10,16650,18500,2023-04-25,세종(세종서적),국내도서,사회문제 일반,사회과학,사회문제
1723,9788937475481,184148396,망할 놈의 예술을 한답시고,민음사 세계시인선 리뉴얼판. 찰스 부코스키가 노년에 마지막으로 출간한 대표작 &lt...,"찰스 부코스키 (지은이), 황소연 (옮긴이)",3317,51168,9,11700,13000,2019-02-22,민음사,국내도서,외국시,소설/시/희곡,시


In [16]:
best_blog['isbn'].value_counts()

0                5
                 2
9791164137114    1
9788937473562    1
9791166055102    1
                ..
9788925536927    1
9791187799023    1
9788935213573    1
9791169472807    1
9791159319181    1
Name: isbn, Length: 1720, dtype: int64

In [17]:
best_blog[(best_blog['isbn']=='0') | (best_blog['isbn'] == '9791192500089')]

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
992,0,48759842,폭풍의 언덕 (한글판 + 영문판),더클래식 세계문학 컬렉션 51권. 서른 해의 짧은 생을 살다 간 에밀리 브론테가 세...,"에밀리 브론테 (지은이), 김명신 (옮긴이)",13713,40078,9,990,990,2014-11-05,더클래식,전자책,서양근대문학,고전,서양고전문학
1180,0,110853184,6월 민주화대투쟁 (1987년 7월 출간),,한국기독교사회문제연구원 (지은이),20821,58331,10,0,0,2020-02-02,민중사,전자책,한국전쟁 이후~현재,역사,한국근현대사
1599,0,145769366,"2018 남북정상회담 : 평화, 새로운 시작",,청와대 (지은이),10650,57860,7,0,0,2020-02-01,알라딘,전자책,한국정치사정/정치사-일반,사회과학,한국정치사정/정치사
1611,0,199734335,클루지 : 생각의 역사를 뒤집는 기막힌 발견 - 생각의 역사를 뒤집는 기막힌 발견,"세계가 주목하는 지성, 뉴욕대학교 심리학과 교수 개리 마커스는 인간의 마음이 '클루...",최호영 (옮긴이),6588,57973,7,9660,9660,2019-07-24,갤리온,전자책,교양 심리학,인문학,심리학/정신분석학
1619,0,250577227,"바뀐 주택임대차보호법 22문22답 - 31년 만의 법개정, 세입자 권리 어떻게 달라...",,"참여연대, 민주사회를 위한 변호사모임 (지은이)",5274,57916,10,0,0,2020-09-01,참여연대,전자책,생활법률 일반,사회과학,법과 생활


## 4.1 New Special(Book)

In [18]:
book_list4 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=ItemNewSpecial&SearchTarget=Book" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list4.append(book_dict)
new_book = pd.DataFrame(book_list4)
new_book = new_book.drop_duplicates()

In [19]:
new_book = pd.merge(new_book, cid, how='left', left_on='CID', right_on='CID')
new_book.index = [i for i in range(1, len(new_book.index) + 1)]
new_book

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9788961964340,316061307,사적인 그림 읽기 - 고요히 치열했던,저자는 언론학과 서양사를 공부한 새내기 연구자이자 세상의 여러 기준에 맞춰 자신을 ...,이가은 (지은이),0,50978,0,16200,18000,2023-05-04,아트북스,국내도서,미술 이야기,예술/대중문화,미술
2,9791192647128,316043293,국공내전 - 신중국과 대만의 탄생,중국 현대사는 내전을 빼놓고 이야기할 수 없다. 군벌 할거의 천하대란을 평정한 일세...,이철의 (지은이),0,1961,0,25200,28000,2023-05-10,앨피,국내도서,중국근현대사(아편전쟁 이후),역사,중국사
3,9788978641197,316034530,"낙원의 몰락, 티베트 이전의 샹그릴라",2004년부터 15년 동안 120여 좌의 티베트 불교사원을 답사하고 80여 명의 유...,심혁주 (지은이),310,51378,0,19800,22000,2023-02-27,산책(도서출판),국내도서,교양 인문학,인문학,교양 인문학
4,9791167961150,316031636,1000만원으로 당장 시작하는 부동산 소액 경매 - 빌라에서 시작해 다가구 월세 건...,부동산 경매 멘토로서 부린이들에게 전수했던 실전 부동산 경매 기술을 빠짐없이 담고 ...,김동하(은행밖 은행원) (지은이),0,175,0,16200,18000,2023-05-10,반니,국내도서,부동산/경매,경제경영,재테크/투자
5,9788932041490,316018772,신발의 눈을 꼭 털어주세요,2010년 『세계의 문학』 신인상을 통해 작품 활동을 시작한 심지아 시인의 두번째 ...,심지아 (지은이),110,51167,0,10800,12000,2023-05-03,문학과지성사,국내도서,한국시,소설/시/희곡,시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,9791185153551,313347870,"믿음을 묻는 딸에게, 아빠가 - 기독교에 회의적인 교양인과 나누고 싶은 질문 25가지","읽고, 고민하고, 실천하는, 지적인 평신도, 한국교회에 없나요? 기독교에 회의적인 ...",정한욱 (지은이),11330,51601,9,16200,18000,2023-03-24,정은문고,국내도서,신앙생활일반,종교/역학,기독교(개신교)
997,9788994606217,307961370,현대 중국의 탄생 - 청제국에서 시진핑까지,,"클라우스 뮐한 (지은이), 윤형진 (옮긴이)",2965,1961,10,46800,52000,2023-03-28,너머북스,국내도서,중국근현대사(아편전쟁 이후),역사,중국사
998,9791160263060,313343408,"땅거미 질 때 샌디에이고에서 로스앤젤레스로 운전하며 소형 디지털 녹음기에 구술한, ...",데뷔 이후 10년 간 독창적인 형식과 언어로 매번 새롭게 주목받아온 작가 정지돈의 ...,정지돈 (지은이),6765,50993,9,12600,14000,2023-03-28,작가정신,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
999,9791188547234,313340779,마더코드,캐럴 스티버스의 글쓰기에 대한 사랑과 과학의 가능성에 대한 흥미를 결합한 소설. 인...,"캐럴 스티버스 (지은이), 공보경 (옮긴이)",980,89481,9,15300,17000,2023-03-15,폴라북스(현대문학),국내도서,외국 과학소설,소설/시/희곡,과학소설(SF)


In [20]:
new_book = new_book[new_book['customer_review_rank'] != 0]
new_book.index = [i for i in range(1, len(new_book.index) + 1)]
new_book

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9788954692526,315992660,각각의 계절,유려하고도 엄정한 문장의 아름다움을 일깨우며 한국문학이 신뢰하는 이름이 된 작가 권...,권여선 (지은이),11510,50993,10,13500,15000,2023-05-07,문학동네,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
2,9788954692342,315799522,북과 남,"제인 오스틴의 계보를 잇는 탁월한 이야기꾼 엘리자베스 개스켈의 사회소설로, “『오...","엘리자베스 개스켈 (지은이), 민승남 (옮긴이)",1280,50919,10,19800,22000,2023-04-28,문학동네,국내도서,영미소설,소설/시/희곡,영미소설
3,9791191328806,315797070,실전 매수매도 기법 - 100만 원으로 시작해 100억대 수익을 올린 슈퍼개미 데이짱의,"매수와 매도 두 관점에서 데이짱이 시장에서 살아남을 수 있었던 기법을 소개하며, 매...",김영옥 (지은이),10540,174,10,24300,27000,2023-06-08,이레미디어,국내도서,주식/펀드,경제경영,재테크/투자
4,9788936439019,315794644,작별 곁에서,소설가 신경숙이 데뷔 38년 만에 첫번째 연작소설 『작별 곁에서』를 출간했다. 예기...,신경숙 (지은이),2410,50993,7,13500,15000,2023-05-03,창비,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
5,9788901271514,315772249,암컷들 - 방탕하고 쟁취하며 군림하는,"마다가스카르의 정글과 케냐의 평원, 하와이나 캐나다의 바다 등을 종횡무진 모험하면서...","루시 쿡 (지은이), 조은영 (옮긴이)",5630,51290,10,19800,22000,2023-05-07,웅진지식하우스,국내도서,동물 일반,과학,동물과 식물
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,9791185153551,313347870,"믿음을 묻는 딸에게, 아빠가 - 기독교에 회의적인 교양인과 나누고 싶은 질문 25가지","읽고, 고민하고, 실천하는, 지적인 평신도, 한국교회에 없나요? 기독교에 회의적인 ...",정한욱 (지은이),11330,51601,9,16200,18000,2023-03-24,정은문고,국내도서,신앙생활일반,종교/역학,기독교(개신교)
625,9788994606217,307961370,현대 중국의 탄생 - 청제국에서 시진핑까지,,"클라우스 뮐한 (지은이), 윤형진 (옮긴이)",2965,1961,10,46800,52000,2023-03-28,너머북스,국내도서,중국근현대사(아편전쟁 이후),역사,중국사
626,9791160263060,313343408,"땅거미 질 때 샌디에이고에서 로스앤젤레스로 운전하며 소형 디지털 녹음기에 구술한, ...",데뷔 이후 10년 간 독창적인 형식과 언어로 매번 새롭게 주목받아온 작가 정지돈의 ...,정지돈 (지은이),6765,50993,9,12600,14000,2023-03-28,작가정신,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
627,9791188547234,313340779,마더코드,캐럴 스티버스의 글쓰기에 대한 사랑과 과학의 가능성에 대한 흥미를 결합한 소설. 인...,"캐럴 스티버스 (지은이), 공보경 (옮긴이)",980,89481,9,15300,17000,2023-03-15,폴라북스(현대문학),국내도서,외국 과학소설,소설/시/희곡,과학소설(SF)


## 4.2 New Special(e-book)

In [21]:
book_list5 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=ItemNewSpecial&SearchTarget=eBook" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['salesPoint'] = item['salesPoint']
        book_dict['CID'] = item['categoryId']
        book_dict['customer_review_rank'] = item['customerReviewRank']
        book_dict['pricesales'] = item['priceSales']
        book_dict['pricestandard'] = item['priceStandard']
        book_dict['pubdate'] = item['pubDate']
        book_dict['publisher'] = item['publisher']

        book_list5.append(book_dict)
new_ebook = pd.DataFrame(book_list5)
new_ebook = new_ebook.drop_duplicates()

In [22]:
new_ebook = pd.merge(new_ebook, cid, how='left', left_on='CID', right_on='CID')
new_ebook.index = [i for i in range(1, len(new_ebook.index) + 1)]
new_ebook

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9788954699419,316059488,경찰 살해자,"북유럽 미스터리의 원점, 경찰소설의 모범 ‘마르틴 베크’ 시리즈의 9번째 작품. 전...","페르 발뢰, 마이 셰발 (지은이), 김명남 (옮긴이)",0,57675,9,9990,11100,2023-05-03,엘릭시르,전자책,기타국가,소설/시/희곡,추리/미스터리소설
2,9791156335344,315986197,바람이 되어 살아낼게,"2014년 4월 16일 아침, 3박 4일간의 수학여행을 떠난 아이들에게 참혹한 비극...",유가영 (지은이),210,57562,10,8400,8400,2023-04-03,다른,전자책,청소년 에세이/시,청소년,청소년 문학
3,9791168730564,315792340,전사들의 노래 - 서지 않는 열차를 멈춰 세우며,2021년 12월 시작된 출근길 지하철 시위의 기원을 더듬어보는 기록이다. 몇십 년...,"홍은전 (지은이), 훗한나 (그림), 비마이너 (기획)",510,57697,7,13500,15000,2023-05-01,오월의봄,전자책,사회문제 일반,사회과학,사회문제
4,9791192836102,315791537,서평가의 독서법 - 분열과 고립의 시대의 책읽기,미치코 가쿠타니의 본격 서평집. 100여 권의 책에 관한 간결하고도 핵심을 찌르는 ...,"미치코 가쿠타니 (지은이), 김영선 (옮긴이)",780,57991,10,13860,13860,2023-03-31,돌베개,전자책,책읽기,인문학,책읽기/글쓰기
5,9791168129443,315779929,늙어감을 사랑하게 된 사람들,"생애문화연구소 옥희살롱의 공동대표 김영옥이 농부, 주거복지 서비스 관리자, 요양보호...",김영옥 (지은이),460,56626,0,11030,12250,2023-04-27,위즈덤하우스,전자책,인문 에세이,인문학,인문 에세이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,9791164051984,310361061,내 인생을 바꾼 거절 - 실패를 자산으로 만든 여성들,"실패를 겪거나 거절을 당한 뒤 다시 일어설 힘이 필요한 이들, 직업 세계로의 진입을...","제시카 배컬 (지은이), 오윤성 (옮긴이)",30,74384,10,12000,12000,2023-02-10,북하우스,전자책,여성의 자기계발,자기계발,여성의 자기계발
175,9788964627303,310339565,반도체 삼국지,"반도체공학자이자 첨단산업 분야의 전략가 권석준 교수가 한국, 일본, 그리고 중국 반...",권석준 (지은이),910,44342,8,14000,14000,2023-02-24,뿌리와이파리,전자책,한국 경제사/경제전망,경제경영,경제학/경제일반
176,9791169794107,309611663,수호자들,"《수호자들》은 《타임 투 킬》, 《펠리컨 브리프》 등 수많은 베스트셀러를 배출하고 ...","존 그리샴 (지은이), 남명성 (옮긴이)",1010,56540,10,12600,12600,2023-02-24,하빌리스(대원씨아이),전자책,영미소설,소설/시/희곡,영미소설
177,9791158748593,309151684,나도 모르는 내 마음의 심리법칙 - 우리는 왜 가끔 미친 짓을 하는 걸까,저자는 다양한 구체적 사례를 통해 독자가 우울증을 이해할 수 있도록 돕는다. 심리적...,"야오야오 (지은이), 김진아 (옮긴이)",165,40177,10,11760,11760,2023-02-10,미디어숲,전자책,인간관계 일반,자기계발,인간관계


In [23]:
new_ebook = new_ebook[new_ebook['customer_review_rank'] != 0]
new_ebook.index = [i for i in range(1, len(new_ebook.index) + 1)]
new_ebook

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
1,9788954699419,316059488,경찰 살해자,"북유럽 미스터리의 원점, 경찰소설의 모범 ‘마르틴 베크’ 시리즈의 9번째 작품. 전...","페르 발뢰, 마이 셰발 (지은이), 김명남 (옮긴이)",0,57675,9,9990,11100,2023-05-03,엘릭시르,전자책,기타국가,소설/시/희곡,추리/미스터리소설
2,9791156335344,315986197,바람이 되어 살아낼게,"2014년 4월 16일 아침, 3박 4일간의 수학여행을 떠난 아이들에게 참혹한 비극...",유가영 (지은이),210,57562,10,8400,8400,2023-04-03,다른,전자책,청소년 에세이/시,청소년,청소년 문학
3,9791168730564,315792340,전사들의 노래 - 서지 않는 열차를 멈춰 세우며,2021년 12월 시작된 출근길 지하철 시위의 기원을 더듬어보는 기록이다. 몇십 년...,"홍은전 (지은이), 훗한나 (그림), 비마이너 (기획)",510,57697,7,13500,15000,2023-05-01,오월의봄,전자책,사회문제 일반,사회과학,사회문제
4,9791192836102,315791537,서평가의 독서법 - 분열과 고립의 시대의 책읽기,미치코 가쿠타니의 본격 서평집. 100여 권의 책에 관한 간결하고도 핵심을 찌르는 ...,"미치코 가쿠타니 (지은이), 김영선 (옮긴이)",780,57991,10,13860,13860,2023-03-31,돌베개,전자책,책읽기,인문학,책읽기/글쓰기
5,9791191891300,315775395,메리골드 마음 세탁소,『하고 싶은 대로 살아도 괜찮아』를 비롯해 약 50만 독자들에게 큰 사랑을 받아온 ...,윤정은 (지은이),1120,56549,9,10500,10500,2023-04-27,북로망스,전자책,판타지/환상문학,소설/시/희곡,판타지/환상문학
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,9791164051984,310361061,내 인생을 바꾼 거절 - 실패를 자산으로 만든 여성들,"실패를 겪거나 거절을 당한 뒤 다시 일어설 힘이 필요한 이들, 직업 세계로의 진입을...","제시카 배컬 (지은이), 오윤성 (옮긴이)",30,74384,10,12000,12000,2023-02-10,북하우스,전자책,여성의 자기계발,자기계발,여성의 자기계발
171,9788964627303,310339565,반도체 삼국지,"반도체공학자이자 첨단산업 분야의 전략가 권석준 교수가 한국, 일본, 그리고 중국 반...",권석준 (지은이),910,44342,8,14000,14000,2023-02-24,뿌리와이파리,전자책,한국 경제사/경제전망,경제경영,경제학/경제일반
172,9791169794107,309611663,수호자들,"《수호자들》은 《타임 투 킬》, 《펠리컨 브리프》 등 수많은 베스트셀러를 배출하고 ...","존 그리샴 (지은이), 남명성 (옮긴이)",1010,56540,10,12600,12600,2023-02-24,하빌리스(대원씨아이),전자책,영미소설,소설/시/희곡,영미소설
173,9791158748593,309151684,나도 모르는 내 마음의 심리법칙 - 우리는 왜 가끔 미친 짓을 하는 걸까,저자는 다양한 구체적 사례를 통해 독자가 우울증을 이해할 수 있도록 돕는다. 심리적...,"야오야오 (지은이), 김진아 (옮긴이)",165,40177,10,11760,11760,2023-02-10,미디어숲,전자책,인간관계 일반,자기계발,인간관계


## 4.3 New Special(Book + ebook)

In [24]:
new_df = pd.concat((new_book, new_ebook))
new_df = new_df.drop_duplicates()
new_df.index = [i for i in range(len(new_df.index))]
new_df

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,9788954692526,315992660,각각의 계절,유려하고도 엄정한 문장의 아름다움을 일깨우며 한국문학이 신뢰하는 이름이 된 작가 권...,권여선 (지은이),11510,50993,10,13500,15000,2023-05-07,문학동네,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
1,9788954692342,315799522,북과 남,"제인 오스틴의 계보를 잇는 탁월한 이야기꾼 엘리자베스 개스켈의 사회소설로, “『오...","엘리자베스 개스켈 (지은이), 민승남 (옮긴이)",1280,50919,10,19800,22000,2023-04-28,문학동네,국내도서,영미소설,소설/시/희곡,영미소설
2,9791191328806,315797070,실전 매수매도 기법 - 100만 원으로 시작해 100억대 수익을 올린 슈퍼개미 데이짱의,"매수와 매도 두 관점에서 데이짱이 시장에서 살아남을 수 있었던 기법을 소개하며, 매...",김영옥 (지은이),10540,174,10,24300,27000,2023-06-08,이레미디어,국내도서,주식/펀드,경제경영,재테크/투자
3,9788936439019,315794644,작별 곁에서,소설가 신경숙이 데뷔 38년 만에 첫번째 연작소설 『작별 곁에서』를 출간했다. 예기...,신경숙 (지은이),2410,50993,7,13500,15000,2023-05-03,창비,국내도서,2000년대 이후 한국소설,소설/시/희곡,한국소설
4,9788901271514,315772249,암컷들 - 방탕하고 쟁취하며 군림하는,"마다가스카르의 정글과 케냐의 평원, 하와이나 캐나다의 바다 등을 종횡무진 모험하면서...","루시 쿡 (지은이), 조은영 (옮긴이)",5630,51290,10,19800,22000,2023-05-07,웅진지식하우스,국내도서,동물 일반,과학,동물과 식물
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,9791164051984,310361061,내 인생을 바꾼 거절 - 실패를 자산으로 만든 여성들,"실패를 겪거나 거절을 당한 뒤 다시 일어설 힘이 필요한 이들, 직업 세계로의 진입을...","제시카 배컬 (지은이), 오윤성 (옮긴이)",30,74384,10,12000,12000,2023-02-10,북하우스,전자책,여성의 자기계발,자기계발,여성의 자기계발
798,9788964627303,310339565,반도체 삼국지,"반도체공학자이자 첨단산업 분야의 전략가 권석준 교수가 한국, 일본, 그리고 중국 반...",권석준 (지은이),910,44342,8,14000,14000,2023-02-24,뿌리와이파리,전자책,한국 경제사/경제전망,경제경영,경제학/경제일반
799,9791169794107,309611663,수호자들,"《수호자들》은 《타임 투 킬》, 《펠리컨 브리프》 등 수많은 베스트셀러를 배출하고 ...","존 그리샴 (지은이), 남명성 (옮긴이)",1010,56540,10,12600,12600,2023-02-24,하빌리스(대원씨아이),전자책,영미소설,소설/시/희곡,영미소설
800,9791158748593,309151684,나도 모르는 내 마음의 심리법칙 - 우리는 왜 가끔 미친 짓을 하는 걸까,저자는 다양한 구체적 사례를 통해 독자가 우울증을 이해할 수 있도록 돕는다. 심리적...,"야오야오 (지은이), 김진아 (옮긴이)",165,40177,10,11760,11760,2023-02-10,미디어숲,전자책,인간관계 일반,자기계발,인간관계


## 5.1 Best + Blog + New Special

In [25]:
len(best_blog), len(new_df)

(1725, 802)

In [26]:
best_blog_new = pd.concat((best_blog, new_df))
best_blog_new = best_blog_new.drop_duplicates()
best_blog_new.index = [i for i in range(len(best_blog_new.index))]
best_blog_new

,isbn,itemid,title,description,author,salesPoint,CID,customer_review_rank,pricesales,pricestandard,pubdate,publisher,mall,category_name,1Depth,2Depth
0,,316015232,주술회전 22 (초판한정 스페셜 일러스트 카드 + 선착순 한정 옷코츠 유타 포토 카...,"사쿠라지마 콜로니에 갑자기 날아든 의문의 주령! 그것은 사후에 저주로 돌변한, 마키...","아쿠타미 게게 (지은이), 이정운 (옮긴이)",234160,4133,10,5400,6000,2023-05-31,서울미디어코믹스(서울문화사),국내도서,액션 판타지,만화,본격장르만화
1,9791168473690,309295168,세이노의 가르침,"2000년부터 발표된 그의 주옥같은 글들. 독자들이 자발적으로 만든 제본서는 물론,...",세이노 (지은이),846030,70216,9,6480,7200,2023-03-02,데이원,국내도서,성공학,자기계발,성공
2,9791168415232,315275569,흔한남매 13,"으뜸이와 에이미의 서프라이즈 장난 대결, 소풍 도시락 싸기 대작전, 신기하고 오싹한...","흔한남매 (지은이), 백난도 (글), 유난희 (그림), 흔한컴퍼니 (감수)",112480,49995,10,13050,14500,2023-04-25,미래엔아이세움,국내도서,만화 일반,어린이,TV/만화/영화
3,9791190299770,313814762,모든 삶은 흐른다,2022년 프랑스 최고의 철학과 교수로 꼽힌 로랑스 드빌레르의 인문에세이로 출간 후...,"로랑스 드빌레르 (지은이), 이주영 (옮긴이)",73175,51378,10,15120,16800,2023-04-03,FIKA(피카),국내도서,교양 인문학,인문학,교양 인문학
4,9791188331888,313250109,사장학개론,"한국과 미국, 전 세계를 오가며 ‘사장을 가르치는 사장’으로 알려진 『돈의 속성』의...",김승호 (지은이),137645,3103,9,22500,25000,2023-04-19,스노우폭스북스,국내도서,경영 일반,경제경영,기업 경영
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2242,9791164051984,310361061,내 인생을 바꾼 거절 - 실패를 자산으로 만든 여성들,"실패를 겪거나 거절을 당한 뒤 다시 일어설 힘이 필요한 이들, 직업 세계로의 진입을...","제시카 배컬 (지은이), 오윤성 (옮긴이)",30,74384,10,12000,12000,2023-02-10,북하우스,전자책,여성의 자기계발,자기계발,여성의 자기계발
2243,9788964627303,310339565,반도체 삼국지,"반도체공학자이자 첨단산업 분야의 전략가 권석준 교수가 한국, 일본, 그리고 중국 반...",권석준 (지은이),910,44342,8,14000,14000,2023-02-24,뿌리와이파리,전자책,한국 경제사/경제전망,경제경영,경제학/경제일반
2244,9791169794107,309611663,수호자들,"《수호자들》은 《타임 투 킬》, 《펠리컨 브리프》 등 수많은 베스트셀러를 배출하고 ...","존 그리샴 (지은이), 남명성 (옮긴이)",1010,56540,10,12600,12600,2023-02-24,하빌리스(대원씨아이),전자책,영미소설,소설/시/희곡,영미소설
2245,9791158748593,309151684,나도 모르는 내 마음의 심리법칙 - 우리는 왜 가끔 미친 짓을 하는 걸까,저자는 다양한 구체적 사례를 통해 독자가 우울증을 이해할 수 있도록 돕는다. 심리적...,"야오야오 (지은이), 김진아 (옮긴이)",165,40177,10,11760,11760,2023-02-10,미디어숲,전자책,인간관계 일반,자기계발,인간관계


In [27]:
best_blog_new['isbn'].value_counts()

0                5
                 4
9791190299886    2
9791138414104    1
9791101825401    1
                ..
9788965424222    1
9791188601233    1
9788937886966    1
9788936438869    1
9791164840601    1
Name: isbn, Length: 2239, dtype: int64

In [33]:
best_blog_new = best_blog_new.drop(best_blog_new[(best_blog_new['isbn'] == '0') | (best_blog_new['isbn'] == '')].index) \
                .reset_index(drop=True)
print(best_blog_new['isbn'].value_counts())
best_blog_new.drop_duplicates(['isbn'], inplace=True, ignore_index=True)
best_blog_new['isbn'].value_counts())

9791168473690    1
9791168999114    1
9791101825401    1
9791168826472    1
9791159038624    1
                ..
9788934989257    1
9788965424222    1
9791188601233    1
9788937886966    1
9791164840601    1
Name: isbn, Length: 2237, dtype: int64 
 2237


(9791168473690    1
 9791168999114    1
 9791101825401    1
 9791168826472    1
 9791159038624    1
                 ..
 9788934989257    1
 9788965424222    1
 9791188601233    1
 9788937886966    1
 9791164840601    1
 Name: isbn, Length: 2237, dtype: int64,
 2237)

In [41]:
best_blog_new = best_blog_new.drop(best_blog_new[best_blog_new['description'] == ''].index) \
                .reset_index(drop=True)
len(best_blog_new)

1804

In [42]:
best_blog_new.to_excel(dir_path + "aladin_20230502.xlsx", index = False, header = True)